In [5]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import os
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,GlobalAvgPool2D
from keras.layers import Conv2D, MaxPooling2D
import keras
from keras.preprocessing import image
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications import xception
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input, decode_predictions
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score


Using TensorFlow backend.


Wczytujemy dane z nazwa pliku i odpowiadajacym im rasom psow:

In [3]:
df = pd.read_csv('/content/drive/My Drive/app/labels.csv')
df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


Przed przystapieniem do klasyfikacji ras psow na podstawie zdjec, pozwolimy sobie na redukcje ich wymiaru, by wyeliminowac skorelowane piksele z naszego modelu. Pozwala to zmniejszyc rozmiar naszego problemu i skrocic czas wykonywania klasyfikacji, ktory bedzie kluczowy dla modelu glebokich sieci.

In [ ]:
import cv2

Jezeli liczba równan jest mniejsza od ilosci niewiadomych nie mozna skorzystac z metody najmniejszych
kwadratów. Istnieje jednak uogólnienie tej metody oparte na twierdzeniu nazywanym twierdzeniem o
rozkladzie macierzy wzgledem wartosci szczególnych zwane tez twierdzeniem dekompozycji widmowej
macierzy.

Twierdzenie

Dowolna macierz $A$ wymiaru $m$ x $n$ mo¿e byæ przedstawiona w postaci 

$$A=U\Sigma V^*,$$

gdzie $U$ i $V$ sa macierzami ortogonalnym, odpowiednio, $U$ wymiaru $m$ x $m$, $V$ wymiaru $n$ x $n$, zaœ $\Sigma$ jest macierza diagonalna wymiaru $m$ x $n$ o nieujemnych wartosciach na przekatnej.

Moze istniec wiecej niz jeden mozliwy rozklad macierzy $A$. Nieujemna wartosc $\sigma$ jest wartoœcia osobliwa $A$, wtedy i tylko wtedy gdy istnieja wektory $u$ i $v$ dlugoœci 1 takie, ze

$$Av=\sigma v$$
oraz
$$A^Tu=\sigma u.$$

Wektor $u$ jest nazywany lewym osobliwym wektorem, a wektor $v$ prawym osobliwym wektorem. Dla kazdej dekompozycji SVD, elementy diagonalne macierzy $A$ sa wartoœciami
osobliwymi $A$, $m$ kolumn macierzy $U$ to odpowiednie lewe osobliwe wektory $A$, a $n$ kolumn macierzy $V$ to prawe osobliwe wektory $A$.


Jest to metoda matematyczna stosowana m.in. w analizie statystycznej sluzaca do redukcji wymiaru macierzy.

W celu redukcji wymiaru obrazow stosujemy SVD opisane powyzej. Pobieramy zatem kazdy plik z naszego zbioru treingowego i rozdzielamy go na kolory (każdy kolor z osobna). Następnie liczymy komponenty dla każdego koloru. W zwiazku z tym, ze nie dysponujemy komputerami o duzych zasobach, podchodzimy do redukcji heurystycznie, to znaczy patrzymy na obraz dla kilku prob przy roznej liczbie komponentow. Ostatecznie wybor padl na redukcje do 50 komponentow. Kolejnym krokiem jest sprawdzenie, czy wartość wyskakuje poza wartosci dla RGB 0-255. Dalej konwertujemy tablicę do obrazu i zapisujemy w pliku.

In [ ]:
from numpy.linalg import svd
i=0
for i in range(0,10500):
    
    path = r'C:\\Python education\\WADProjects\\Proj1\\train\\' + data['id'][i] +'.jpg'
    img = np.asarray(Image.open(path))

    
        r = img[:,:,0]  # array for R
    g = img[:,:,1]  # array for G
    b = img[:,:,2] # array for B
    

    
        ur,sr,vr = svd(r, full_matrices=False)
    ug,sg,vg = svd(g, full_matrices=False)
    ub,sb,vb = svd(b, full_matrices=False)
    
    
    k=50

    
    rr = np.dot(ur[:,:k],np.dot(np.diag(sr[:k]), vr[:k,:]))
    rg = np.dot(ug[:,:k],np.dot(np.diag(sg[:k]), vg[:k,:]))
    rb = np.dot(ub[:,:k],np.dot(np.diag(sb[:k]), vb[:k,:]))
    

    rimg = np.zeros(img.shape)
    

    rimg[:,:,0] = rr
    rimg[:,:,1] = rg
    rimg[:,:,2] = rb
    
  

    for ind1, row in enumerate(rimg):
        for ind2, col in enumerate(row):
            for ind3, value in enumerate(col):
                if value < 0:
                    rimg[ind1,ind2,ind3] = abs(value)
                if value > 255:
                    rimg[ind1,ind2,ind3] = 255

               
    compressed_image = rimg.astype(np.uint8)
    
    
    compressed_image = Image.fromarray(compressed_image)
    compressed_image.save(r'C:\\Python education\\WADProjects\\Proj1\\new_train_color_prawidlowy\\' + data['id'][i] +'.jpg')
   
    print(i)
    
 

Ponizej widzimy rezultat redukcji wymiaru: z lewej - oryginal, z prawej - redukcja do 50 komponentow dla kazdego z kolorow.

In [12]:
from PIL import Image
image = Image.open('C:\\Python education\\WADProjects\\Proj1\\new_train_color_prawidlowy\\0a0c223352985ec154fd604d7ddceabd.jpg')
image2 = Image.open('C:\\Python education\\WADProjects\\Proj1\\train\\0a0c223352985ec154fd604d7ddceabd.jpg')

f, axarr = plt.subplots(1,2,figsize=(300,300))

#image.show()
axarr[0].imshow(np.asarray(image2))
axarr[1].imshow(np.asarray(image))

Tworzymy zmienną w celu wyciągnięcia nazwy plikow zbioru testowego(przykładowe sample submission)

In [ ]:
data_test=pd.read_csv(r'C:\\Python education\\WADProjects\\Proj1\\sample_submission.csv')

Za pomocą SVD redukujemy wymiary dla zbioru testowego jak zbiorze treningowym  

In [ ]:
from numpy.linalg import svd

for i in range(0,10500):
    k=50

    path = r'C:\\Python education\\WADProjects\\Proj1\\test\\' + data_test['id'][i] +'.jpg'
    img = np.asarray(Image.open(path))

    

    r = img[:,:,0] 
    g = img[:,:,1] 
    b = img[:,:,2] 
    

    

    ur,sr,vr = svd(r, full_matrices=False)
    ug,sg,vg = svd(g, full_matrices=False)
    ub,sb,vb = svd(b, full_matrices=False)
    

    rr = np.dot(ur[:,:k],np.dot(np.diag(sr[:k]), vr[:k,:]))
    rg = np.dot(ug[:,:k],np.dot(np.diag(sg[:k]), vg[:k,:]))
    rb = np.dot(ub[:,:k],np.dot(np.diag(sb[:k]), vb[:k,:]))

    rimg = np.zeros(img.shape)
    

    rimg[:,:,0] = rr
    rimg[:,:,1] = rg
    rimg[:,:,2] = rb
    

    for ind1, row in enumerate(rimg):
        for ind2, col in enumerate(row):
            for ind3, value in enumerate(col):
                if value < 0:
                    rimg[ind1,ind2,ind3] = abs(value)
                if value > 255:
                    rimg[ind1,ind2,ind3] = 255


    compressed_image = rimg.astype(np.uint8)

    compressed_image = Image.fromarray(compressed_image)
    compressed_image.save(r'C:\\Python education\\WADProjects\\Proj1\\new_test_color\\' + data_test['id'][i] +'.jpg')

    print(i)

W trakcie trwania projektu stanelismy przed problemem zasobow naszych komputerow. Do klasyfikacji wykorzystalismy glebokie sieci neuronowe, jednak nie spodziewalismy sie, ze praca na CPU moze byc nie wystarczajaca. TensorFlow wspiera prace na graficznym procesorze GPU, ale... 

<i>"The GPU-enabled version of TensorFlow has several requirements such as 64-bit Linux, Python 2.7 (or 3.3+ for Python 3), NVIDIA CUDA 7.5 (CUDA 8.0 required for Pascal GPUs) and NVIDIA, cuDNN v4.0 (minimum) or v5.1 (recommended)."</i>

Jednakze nikt nie posiadal takiej karty graficznej. Warto wspomniec, ze z pomoca przychodzi tutaj Google. Okazuje sie, ze wystarczy do tego tylko konto na Google Drive. Wystarczy jedynie zainstalowac wtyczke Colaboratory, ktora pozwala obslugiwac pliki o rozszerzeniu .ipynb, zachowujac sie tak samo jak Jupyter lokalnie. Wiecej informacji jak uzywac tej wtyczki znajdziemy tutaj: https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d. Pozwolimy zacytowac sobie podsumowanie autora tego artykulu:

<i>"I think Colab will bring a new breath to Deep Learning and AI studies all over the world."</i> - mial racje.

Montujemy nasz google drive, tak by Colab mogl odczytywac pliki zawarte na naszym dysku Google.

In [2]:
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Konwertujemy wektor ras psów na macierz, której każdy z wierszy jest przedstawieniem rasy psa w ujęciu zero-jedynkowym.

In [0]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat = le.fit_transform(df.breed)
from keras.utils.np_utils import to_categorical
mat = to_categorical(cat)

Przygotowujemy funkcje do zmniejszania rozmiar zdjęć z pliku do rozmiaru 150x150

In [0]:
import cv2
def readImgResize(name,path):
    img = cv2.imread(path+name)
    img = cv2.resize(img,(150,150))
    return image.img_to_array(img)

Zmniejszamy wszystkie zdjecia ze zbioru treningowego do jednego rozmiaru 150x150.

In [21]:
training_data = np.zeros(shape=(len(df.id),150,150,3))
for i,j in enumerate(df.id):
    training_data[i]=readImgResize(j+".jpg",path=r'/content/drive/My Drive/app/new_train_color_prawidlowy/')
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

Przygotowujemy zbior treningowy do etapu uczenia modelu

In [0]:
X_train, X_test, y_train, y_test = train_test_split( training_data, mat, test_size=0.05, random_state=11)

Zajmiemy się teraz trenowaniem modelu. Korzystamy z modelu VGG19, w którym sami definiujemy ostatnią warstwę. Dodaną warstwę spłaszczamy do postaci tablicy jednowymiarowej. Dalej dodajemy nową warstwę neuronów posiadającycą 120 węzłów. Funkcją aktywacji jest tu funkcja "softmax", ktora sluzy lepiej do zagadnienia multiklasyfikacji (posiadamy 120 klas) anizeli tradycyjny "sigmoid". Trening przeprowadzamy wyłącznie na górnej warstwie z prędkością uczenia (lr) równą 0.00001.

In [23]:
from keras.applications import InceptionV3
from keras.models import Model
from keras.optimizers import adam

num_class = 120
im_size = 150

base_model = VGG19(weights='imagenet', include_top=False, input_shape=(im_size, im_size, 3))

x = base_model.output
x = Flatten()(x)
predictions = Dense(num_class, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)


for layer in base_model.layers:
    layer.trainable = False
model.compile(adam(lr=0.00001),loss='categorical_crossentropy', 
              metrics=["accuracy"])

callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
model.summary()

80142336/80134624 [==============================] - 1s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool

Usuwamy poprzednie zmienne w celu zwolnienia pamieci do uczenia sieci glebokich. Dane te byly przeznaczone tylko do przygotowania zbioru treningowego do uczenia. Zostaly one uzyte w momencie przygotowywania zbioru treningowego, zatem mozemy je usunac i korzystac z przygotowanego juz zbioru. 

In [0]:
del training_data, mat

Warto zaznaczyc, ze proces uczenia sieci glebokich pochlania wiele zasobow komputera. Przy podejsciu uczenia bez wykorzystania procesora graficznego (GPU), a z uzyciem jedynie CPU dla 50 epok zajelo to nam okolo 65 godzin. Z uzyciem GPU na Google Colab zajelo to okolo poltorej godziny. Warto zatem zapisac nauczony model do pliku i wczytywac go w razie potrzeby anizeli uczyc model od nowa.

In [0]:
#from keras.models import load_model
#model = load_model('/content/drive/My Drive/app/breed_vgg19_50epochs.h5')

Uczymy model przygotowanym zbiorem treningowym.

In [25]:
model.fit(X_train/255, y_train, epochs=50, validation_data=(X_test/255, y_test), verbose=1)
model.save('/content/drive/My Drive/app/breed_vgg19_50epochs.h5')

Train on 9710 samples, validate on 512 samples
Epoch 1/50
9710/9710 [==============================] - 73s 8ms/step - loss: 4.9426 - acc: 0.0083 - val_loss: 4.8273 - val_acc: 0.0117
Epoch 2/50
9710/9710 [==============================] - 64s 7ms/step - loss: 4.7203 - acc: 0.0200 - val_loss: 4.7451 - val_acc: 0.0176
Epoch 3/50
9710/9710 [==============================] - 64s 7ms/step - loss: 4.6175 - acc: 0.0369 - val_loss: 4.6773 - val_acc: 0.0254
Epoch 4/50
9710/9710 [==============================] - 65s 7ms/step - loss: 4.5231 - acc: 0.0533 - val_loss: 4.6168 - val_acc: 0.0391
Epoch 5/50
9710/9710 [==============================] - 76s 8ms/step - loss: 4.4329 - acc: 0.0715 - val_loss: 4.5629 - val_acc: 0.0508
Epoch 6/50
9710/9710 [==============================] - 111s 11ms/step - loss: 4.3483 - acc: 0.0945 - val_loss: 4.5037 - val_acc: 0.0625
Epoch 7/50
9710/9710 [==============================] - 71s 7ms/step - loss: 4.2667 - acc: 0.1160 - val_loss: 4.4530 - val_acc: 0.0742
Epoch 

Widzimy, ze od 35 epoki nasz model zaczyna sie przetrenowywac. Swiadczy o tym wartosc val_acc, ktora zaczela sie utrzymywac na poziomie 19% i przestala znaczaco rosnac (na poziomie 50. epoki wartosc jest 19.34%). Oznacza to tyle, ze wplyw uczenia sie i coraz lepsze rozpoznawanie ras na zbiorze treningowym nie ma przelozenia na rozpoznawanie ras zbioru testowego wydzielonego jako jakas probka ze zbioru treningowego. To motywuje, zeby nie trenowac juz dalej modelu i przechodzimy do predykcji.

Tworzymy liste zdjec ze zbioru testowgo i zmieniamy na wspolny rozmiar 150x150 tak jak poprzednio dla zbioru treningowego, po czym dokonujemy predykcji.

In [0]:
from keras.models import load_model
from datetime import datetime
te = os.listdir(r'/content/drive/My Drive/app/test/')
te_in = np.zeros((len(te),150,150,3))

for num , i in enumerate(te):
    img = readImgResize(i,path=r'/content/drive/My Drive/app/test/')/255
    te_in[num]=img
    print(datetime.now())

    
pred = model.predict(te_in)

2019-01-24 07:12:11.290826
2019-01-24 07:12:12.158343
2019-01-24 07:12:12.604812
2019-01-24 07:12:13.085426
2019-01-24 07:12:13.520755
2019-01-24 07:12:13.956511
2019-01-24 07:12:14.450128
2019-01-24 07:12:14.917967
2019-01-24 07:12:15.433455
2019-01-24 07:12:15.898425
2019-01-24 07:12:16.388898
2019-01-24 07:12:16.897786
2019-01-24 07:12:17.364345
2019-01-24 07:12:17.840113
2019-01-24 07:12:18.291114
2019-01-24 07:12:18.787468
2019-01-24 07:12:19.276253
2019-01-24 07:12:19.692886
2019-01-24 07:12:20.147062
2019-01-24 07:12:20.638776
2019-01-24 07:12:21.082779
2019-01-24 07:12:21.599790
2019-01-24 07:12:22.119472
2019-01-24 07:12:22.569687
2019-01-24 07:12:23.154738
2019-01-24 07:12:23.496131
2019-01-24 07:12:23.977279
2019-01-24 07:12:24.505665
2019-01-24 07:12:24.984888
2019-01-24 07:12:25.463591
2019-01-24 07:12:25.894336
2019-01-24 07:12:26.377036
2019-01-24 07:12:26.874617
2019-01-24 07:12:27.418626
2019-01-24 07:12:27.841109
2019-01-24 07:12:28.305029
2019-01-24 07:12:28.743518
2

Tworzymy ramke danych, ktora spelnia wymagania kaggle do wyslania...

In [0]:
submission = pd.DataFrame(pred , columns =le.classes_.tolist())
submission["id"]=[i.split(".")[0] for i in os.listdir("/content/drive/My Drive/app/new_test_color")]
submission = submission[["id"]+submission.columns[:-1].tolist()]

... i zapisujemy do pliku.

In [0]:
submission.sort_values(by=['id'])
submission.to_csv(r"/content/drive/My Drive/app/nadziejaumieraostatnia.csv",index = False)

Miejsce: 977, Blad: 3.51145